# A quick look at exploring Wintap data

This notebook is a simple example of looking at Wintap data in Jupyter.

It shows how to use:
* SQL via DuckDB
* Basic queries and data review
* Charting data with Altair

In [ ]:
# Install packages we'll need
!pip install duckdb==0.10.1 magic_duckdb altair
%reload_ext magic_duckdb

In [ ]:
IN_COLAB = True
try:
  import google.colab
except:
  IN_COLAB = False

if IN_COLAB:
  # Download the sample data file. Its actually in https://tinyurl.com/wintapdata/ACME-workshop-20231109-20231111.db, but this is a direct reference that works better.
  !gdown 1ELBBx3p2ngVSIyf7Neu0F50vd-qf9XDO

In [ ]:
# Connect to the database instance
# This cell is creating a python variable database connection and then supplying that to the "magic_duckdb" Jupyter extension
# which allows the single-line SQL (%dql) and multiline SQL (%%dql) "magics". https://github.com/iqmo-org/magic_duckdb
import duckdb

conn = duckdb.connect('ACME-workshop-20231109-20231111.db', read_only=True)
%dql -co conn

# Display Data

A few immediate questions are: what tables are here? What columns are in those tables and how do I see some of the data?

In [ ]:
# Lets see all those tables
%dql show all tables

In [ ]:
# Duckdb's summarize command display the schema of a table along with some useful metrics. And its fast!
%dql summarize process

In [ ]:
# Lets see all rows in a table
%dql select * from host

In [ ]:
# Lets just see a few to get an idea of whats in there
%dql select * from process limit 20

# Charting data with Altair

Altair is sophisticated, declaritive python package that allows for large range of charting types that are easily customizable and interactive.

The first thing we need to do to leverage them is convert data from its SQL source to a Pandas dataframe. Pandas are the defacto python data model for just about all data analytics packages.

The Pandas conversion is easily done from DuckDB with the "-o" output to variable flag.


In [ ]:
import altair as alt

%dql -o procDF select process_name, count(*) num_rows from process group by all

In [ ]:
alt.Chart(procDF).mark_bar().encode(
    x='process_name',
    y=alt.Y('num_rows',scale=alt.Scale(type="log")),
    color='process_name',
    tooltip=['process_name:N','num_rows:Q'],
).properties(
    width=1400,
    height=600,
    title="Distribution of process executions by name"
).interactive()


# A more complex example
In this example, we're going to generate a dataframe that will be process utilization per time interval and chart that. For example, in a given 5 minute window, what is the relative percentage of bytes attributed to each process that did network transfers in that period.

Note that many features are calculate for each interval. Try charting different features to see if there is anything intersting beyong the bytes transferred.

In [ ]:
# Define parameters for the next chart
alt.data_transformers.disable_max_rows()
interval='5 minutes'
hostname='ACME-HH-AKA'

In [ ]:
%%dql -j -o netuse_df
SELECT
  -- Get all columns from the inline view
  *,
  -- Using window functions, calculate the percent and rank by time chunk and time chunk rank by host overall activity
  round((tot_bytes/(sum(tot_bytes) OVER (PARTITION BY hostname, time_chunk)))*100,2) tc_process_pct,
  RANK() OVER (PARTITION BY hostname, time_chunk ORDER BY tot_bytes DESC) tc_rank_pos,
  round((tot_bytes/sum(tot_bytes) OVER (PARTITION BY hostname))*100,2) process_pct,
  -- Rank over the host
  RANK() OVER (PARTITION BY hostname ORDER BY tot_bytes DESC) rank_pos
FROM (
-- This query groups detail rows into time buckets, which then used to generate a histogram.
	SELECT
	  p.hostname,
	  p.process_name,
      time_bucket(INTERVAL {{interval}}, pnc.first_seen) time_chunk,
	  count(DISTINCT p.pid_hash) num_pid_hash,
	  count(DISTINCT conn_id) num_conn_id,
	  count(DISTINCT remote_ip_addr) num_remote_ip,
	  count(DISTINCT remote_port) num_remote_port,
	  sum(pnc.total_events) total_events,
	  sum(pnc.total_size) tot_bytes
	FROM process_net_conn pnc
	JOIN process p ON p.pid_hash=pnc.pid_hash
	-- Ignore localhost network activity
	WHERE --pnc.local_ip_addr<>pnc.remote_ip_addr
  	 p.hostname = '{{hostname}}'
  	AND pnc.hostname = '{{hostname}}'
	GROUP BY ALL
) pnc_inner
ORDER BY hostname, time_chunk, tot_bytes DESC

In [ ]:
alt.Chart(netuse_df).mark_bar().encode(
    x='time_chunk',
    y=alt.Y('tot_bytes'), #,scale=alt.Scale(type="symlog")),
    color='process_name',
    tooltip=['process_name:N','tot_length:Q','tc_process_pct:Q','tc_rank_pos:Q','time_chunk:T']
).properties(
    title='Network Activity by Process',
    width=1200,
    height=400
).interactive()